# Module 02: Models & Databases

**Estimated Time:** 2 hours  
**Difficulty:** ⭐

---

## Prerequisites

Before starting this module, ensure you've completed:

- ✅ **Module 00**: Setup & Introduction to Django
- ✅ **Module 01**: Django Basics & First Project
- ✅ You have a Django project created (`myblog`)
- ✅ You have a Django app created (`blog`)
- ⚠️ **Basic understanding of databases** (tables, rows, columns) is helpful

---

## Learning Objectives

By the end of this module, you will:

- ✅ Understand Django's ORM (Object-Relational Mapping)
- ✅ Define models and field types
- ✅ Create and apply database migrations
- ✅ Use the QuerySet API to query data
- ✅ Create model relationships (ForeignKey, ManyToMany)
- ✅ Use Django shell for interactive database work

---

## 1. What is Django ORM?

**ORM (Object-Relational Mapping)** lets you interact with databases using Python code instead of SQL.

### Without ORM (Raw SQL)
```sql
SELECT * FROM blog_post WHERE published = True ORDER BY created_date DESC;
```

### With Django ORM (Python)
```python
Post.objects.filter(published=True).order_by('-created_date')
```

### Benefits
- Write Python, not SQL
- Database-agnostic (switch databases easily)
- Protection against SQL injection
- Automatic schema creation and migrations

## 2. Creating Your First Model

A **model** is a Python class that represents a database table. Let's create a `Post` model for our blog.

In [ ]:
# First, let's set up our paths
import os
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_path = notebook_dir.parent / "projects" / "myblog"
models_file = project_path / "blog" / "models.py"

print(f"Project path: {project_path}")
print(f"Models file: {models_file}")
print(f"Models file exists: {models_file.exists()}")

In [ ]:
# Create Post model
model_code = '''from django.db import models
from django.contrib.auth.models import User
from django.utils import timezone


class Post(models.Model):
    """Blog post model"""
    
    # Status choices
    STATUS_CHOICES = [
        ('draft', 'Draft'),
        ('published', 'Published'),
    ]
    
    # Fields
    title = models.CharField(max_length=200, help_text="Post title")
    slug = models.SlugField(max_length=200, unique=True, help_text="URL-friendly version of title")
    author = models.ForeignKey(User, on_delete=models.CASCADE, related_name='blog_posts')
    content = models.TextField(help_text="Post content")
    created_date = models.DateTimeField(auto_now_add=True)
    updated_date = models.DateTimeField(auto_now=True)
    publish_date = models.DateTimeField(default=timezone.now)
    status = models.CharField(max_length=10, choices=STATUS_CHOICES, default='draft')
    
    class Meta:
        ordering = ['-publish_date']
        verbose_name = 'Post'
        verbose_name_plural = 'Posts'
    
    def __str__(self):
        return self.title
    
    def get_absolute_url(self):
        from django.urls import reverse
        return reverse('post_detail', args=[self.slug])
'''

# Write to models.py
with open(models_file, "w") as f:
    f.write(model_code)

print("✓ Post model created in blog/models.py")
print("\nModel code:")
print(model_code)

### Understanding the Model

#### Field Types
- **CharField**: Short text (max_length required)
- **SlugField**: URL-friendly text
- **TextField**: Long text
- **DateTimeField**: Date and time
- **ForeignKey**: Relationship to another model

#### Field Options
- **max_length**: Maximum character length
- **unique**: Ensure no duplicates
- **default**: Default value
- **auto_now_add**: Set on creation
- **auto_now**: Update on every save
- **choices**: Limit to specific values
- **help_text**: Help text for forms

#### Meta Class
- **ordering**: Default sort order
- **verbose_name**: Human-readable name

#### Methods
- **__str__()**: String representation
- **get_absolute_url()**: Canonical URL

## 3. Creating Migrations

Migrations are Django's way of propagating changes to your models into the database schema.

In [ ]:
# Create migrations
import subprocess

result = subprocess.run(
    ["python", "manage.py", "makemigrations", "blog"],
    capture_output=True,
    text=True,
    cwd=project_path,
)

print(result.stdout)
if result.returncode == 0:
    print("\n✓ Migrations created successfully!")
else:
    print(f"\n✗ Error: {result.stderr}")

In [ ]:
# Apply migrations
result = subprocess.run(
    ["python", "manage.py", "migrate", "blog"], capture_output=True, text=True, cwd=project_path
)

print(result.stdout)
if result.returncode == 0:
    print("\n✓ Database updated successfully!")
else:
    print(f"\n✗ Error: {result.stderr}")

## 4. Django Shell - Interactive Database Work

The Django shell is an interactive Python environment with Django context loaded.

**Note**: The following demonstrates shell commands. In practice, you'd run these in a terminal with:
```bash
python manage.py shell
```

## 5. QuerySet API - Retrieving Data

Let's learn how to query the database using Django's QuerySet API.

### Common QuerySet Methods

| Method | Description | Example |
|--------|-------------|--------|
| `.all()` | Get all objects | `Post.objects.all()` |
| `.filter()` | Filter objects | `Post.objects.filter(status='published')` |
| `.exclude()` | Exclude objects | `Post.objects.exclude(status='draft')` |
| `.get()` | Get single object | `Post.objects.get(id=1)` |
| `.first()` | Get first object | `Post.objects.first()` |
| `.last()` | Get last object | `Post.objects.last()` |
| `.count()` | Count objects | `Post.objects.count()` |
| `.exists()` | Check existence | `Post.objects.filter(title='Test').exists()` |
| `.order_by()` | Sort results | `Post.objects.order_by('-created_date')` |

### Field Lookups

```python
# Exact match
Post.objects.filter(title='My Post')

# Case-insensitive match
Post.objects.filter(title__iexact='my post')

# Contains
Post.objects.filter(title__contains='Django')

# Starts with
Post.objects.filter(title__startswith='How to')

# Greater than
Post.objects.filter(created_date__gt=some_date)

# Less than or equal
Post.objects.filter(created_date__lte=some_date)

# In list
Post.objects.filter(status__in=['draft', 'published'])
```

## 6. Model Relationships

Let's add a `Category` model and create relationships.

In [ ]:
# Add Category model
updated_model_code = '''from django.db import models
from django.contrib.auth.models import User
from django.utils import timezone


class Category(models.Model):
    """Blog category model"""
    name = models.CharField(max_length=100, unique=True)
    slug = models.SlugField(max_length=100, unique=True)
    description = models.TextField(blank=True)
    
    class Meta:
        verbose_name_plural = 'Categories'
        ordering = ['name']
    
    def __str__(self):
        return self.name


class Post(models.Model):
    """Blog post model"""
    
    STATUS_CHOICES = [
        ('draft', 'Draft'),
        ('published', 'Published'),
    ]
    
    title = models.CharField(max_length=200)
    slug = models.SlugField(max_length=200, unique=True)
    author = models.ForeignKey(User, on_delete=models.CASCADE, related_name='blog_posts')
    categories = models.ManyToManyField(Category, related_name='posts', blank=True)
    content = models.TextField()
    created_date = models.DateTimeField(auto_now_add=True)
    updated_date = models.DateTimeField(auto_now=True)
    publish_date = models.DateTimeField(default=timezone.now)
    status = models.CharField(max_length=10, choices=STATUS_CHOICES, default='draft')
    views = models.IntegerField(default=0)
    
    class Meta:
        ordering = ['-publish_date']
    
    def __str__(self):
        return self.title
'''

with open(models_file, "w") as f:
    f.write(updated_model_code)

print("✓ Models updated with Category and relationships")

### Relationship Types

#### ForeignKey (Many-to-One)
```python
author = models.ForeignKey(User, on_delete=models.CASCADE)
```
- Many posts can have the same author
- `on_delete=CASCADE`: Delete posts when user is deleted
- `related_name`: Access posts from user: `user.blog_posts.all()`

#### ManyToManyField
```python
categories = models.ManyToManyField(Category)
```
- Post can have multiple categories
- Category can have multiple posts
- Django creates intermediate table automatically

#### OneToOneField
```python
profile = models.OneToOneField(User, on_delete=models.CASCADE)
```
- One-to-one relationship
- Example: User profile, settings

In [ ]:
# Create and apply new migrations
result = subprocess.run(
    ["python", "manage.py", "makemigrations"], capture_output=True, text=True, cwd=project_path
)
print(result.stdout)

result = subprocess.run(
    ["python", "manage.py", "migrate"], capture_output=True, text=True, cwd=project_path
)
print(result.stdout)
print("\n✓ New migrations applied!")

## 7. Summary & Next Steps

### What We Learned

✅ Django ORM basics  
✅ Creating models with various field types  
✅ Understanding migrations (makemigrations and migrate)  
✅ Using QuerySet API to query data  
✅ Model relationships (ForeignKey, ManyToMany)  
✅ Django shell for interactive work  

### Models We Created

1. **Post**: Blog posts with title, content, author, categories
2. **Category**: Post categories

### Quick Check

Before moving on, ensure you can answer:

1. What does ORM stand for and why is it useful?
2. What command creates migrations? What command applies them?
3. What's the difference between ForeignKey and ManyToManyField?
4. How do you retrieve all published posts using the QuerySet API?
5. What method on a model provides its string representation?

### What's Next

In **Module 03**, we'll:
- Access the Django admin interface
- Register our models
- Customize admin displays
- Add filters, search, and actions

---

**Excellent work! Your database is ready. Continue to Module 03!** 🎉